# Project Title: The Impact of News on the Market
### •	Team Members:
##### 	Rachel Torres, Christian Attard, Jess Alcalde, Nitin Khade
### •	Project Description/Outline:
##### -	We will look at news data and stock data to determine the effects of the news on how the market behaves.
### •	Research Questions to Answer:
##### -	How do news headlines affect the stock market?
##### -	Is there any correlation between certain types of headlines and effects on the market?
##### -	Does negative news affect stocks greater than positive or neutral?
##### -	Can we assign a factor(weighting) to it?

In [1]:
# import dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from news_api import api_key
from x_api import x_api_key

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

# pip install vaderSentiment


### We'll query the news api to gather news headlines from the web

In [3]:
# parameters for news api
q='politics',
from_param='2019-05-24',
language='en',
sort_by='relevancy',
page = 4,
pageSize = 50

url = f'https://newsapi.org/v2/everything?q={q}&from={from_param}&language={language}&sortBy={sort_by}&pageSize={pageSize}&page={page}&apiKey={api_key}'
response = requests.get(url)

url_2 = f'https://newsapi.org/v2/everything?q=apple&from=2019-05-24&to=2019-06-23&language=en&sortBy=popularity&pageSize={pageSize}&page=3&apiKey={api_key}'
response = requests.get(url_2)
# Generate response into json 

In [4]:
pprint(response.json())

{'code': 'maximumResultsReached',
 'message': 'You have requested too many results. Developer accounts are '
            'limited to a max of 100 results. You are trying to request '
            'results 100 to 150. Please upgrade to a paid plan if you need '
            'more results.',
 'status': 'error'}


### We'll use a different API to gather news data since we have a restriction on the news api

In [5]:
# Query contextual web search API
# Replace the following string value with your valid X-RapidAPI-Key.
Your_X_RapidAPI_Key = x_api_key;

# The query parameters: (update according to your search query)
q = "Donald%20Trump%20News" #the search query
pageNumber = 1 #the number of requested page
pageSize = 50 #the size of a page
autoCorrect = True #autoCorrectspelling
safeSearch = False #filter results for adult content


# test response to query and get count of total items and pages
response_test=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, pageNumber, pageSize, autoCorrect,safeSearch),
headers={
"X-RapidAPI-Key": Your_X_RapidAPI_Key
}
).json()

#Get the number of items returned
totalCount = response_test["totalCount"];
totalPages = round(totalCount/pageSize)


In [6]:
pprint(response_test)

{'_type': 'news',
 'didUMean': '',
 'relatedSearch': ['president',
                   'iran',
                   'washington',
                   'white house',
                   'uk',
                   'world <b>news</b>',
                   'fox',
                   'abc',
                   'mexico',
                   'views',
                   'one  page',
                   'took in as much'],
 'totalCount': 2479,
 'value': [{'datePublished': '2019-06-12T21:19:00',
            'description': 'President <b>Donald Trump</b> granted ABC '
                           '<b>News</b> an exclusive, wide-ranging interview '
                           'over the course of two days.',
            'image': {'base64Encoding': None,
                      'height': 558,
                      'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=137322115715991574',
                      'thumbnailHeight': 139,
                      'thumbnailWidth': 247,
                      'u

                      'url': 'https://www.jpost.com/HttpHandlers/ShowImage.ashx?id=370634',
                      'width': 898},
            'isSafe': True,
            'keywords': 'president donald trump,nbc news',
            'language': 'en',
            'provider': {'name': 'jpost'},
            'title': '<b>Trump</b>: "may" talk to Putin at G20 about Russian '
                     'meddling in 2020 election',
            'url': 'https://www.jpost.com/Breaking-News/Trump-may-talk-to-Putin-at-G20-about-Russian-meddling-in-2020-election-NBC-593419'},
           {'datePublished': '2019-06-23T14:05:47',
            'description': 'President <b>Donald Trump</b> told NBC <b>News</b> '
                           'on Sunday that he did not send a message to Tehran '
                           'warning Iran of a U.S. attack, which he later '
                           'called off. Iranian sources told Reuters that '
                           '<b>Trump</b> had warned Tehran via Oman that a 

In [7]:
# generate sample response to look at headers
pprint((response_test['value'][2]))

{'datePublished': '2019-06-24T23:25:57',
 'description': 'U.S. President <b>Donald Trump</b> has no plans to meet with '
                'North Korean leader Kim Jong Un when he travels to South '
                'Korea over the weekend after attending a summit of the Group '
                'of 20 leaders in Japan, a senior administration official said '
                'on Monday.',
 'image': {'base64Encoding': None,
           'height': 798,
           'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=4124172507668342959',
           'thumbnailHeight': 224,
           'thumbnailWidth': 336,
           'url': 'https://s2.reutersmedia.net/resources/r/?m=02&d=20190624&t=2&i=1401173160&w=1200&r=LYNXNPEF5N20G',
           'width': 1200},
 'isSafe': True,
 'keywords': 'president donald trump,updated,news,kim',
 'language': 'en',
 'provider': {'name': 'reuters'},
 'title': "<b>Trump</b> has no plans to meet with North Korea's Kim during "
          'South Korea visit',


In [8]:
print(f' There are {totalPages} pages, with {totalCount} total articles.')

 There are 50 pages, with 2479 total articles.


In [12]:
# create empty lists to hold variable results
url_list = []
title_list = []
description_list = []
keywords_list = []
provider_list = []
date_list = []

# Load news data into pandas dataframe
# page = 1
for page in range(1,totalPages+1):
    response=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, page, pageSize, autoCorrect,safeSearch),
headers={"X-RapidAPI-Key": Your_X_RapidAPI_Key}).json()

# print(response)
    try:
        #Go over each resulting item
        for webPage in response["value"]:

#Get the web page metadata
            url = webPage["url"]
            title = webPage["title"]
            description = webPage["description"]
            keywords = webPage["keywords"]
            provider = webPage["provider"]["name"]
            datePublished = webPage["datePublished"]
        
        
            url_list.append(url)
            title_list.append(title)
            description_list.append(description)
            keywords_list.append(keywords)
            provider_list.append(provider)
            date_list.append(datePublished)

            print(f'printing {title}')    
    except (KeyError,IndexError):
            print('Not found, skipping')


printing ABC <b>News</b> travels with President <b>Donald Trump</b> from Iowa to the White House
printing <b>Donald Trump News</b>
printing <b>Trump</b> has no plans to meet with North Korea's Kim during South Korea visit
printing US, China must compromise to reach deal: Chinese official, East Asia <b>News</b> & Top Stories
printing US-Iran tensions raise threat of an even graver conflict
printing WRAPUP 3-Trump to visit S.Korea as Pompeo raises hope for new N.Korea talks after letter
printing <b>Trump</b> Imposes New US Sanctions on Iran, Targeting Ayatollah Khamenei
printing <b>Trump</b> signs order imposing sanctions on Iran supreme leader  Pasadena Star <b>News</b>
printing New US sanctions on Iran target supreme leader  BBC <b>News</b>
printing <b>Trump</b> imposes new sanctions on Iran, targets supreme leader
printing <b>Trump</b> signs executive order imposing sanctions on Iran
printing <b>Trump</b> announces hard hitting new sanctions against Iran  WSVN 7News
printing <b>Trump<

printing <b>Trump</b> Talks Trade With Apple CEO Cook as China Dispute Looms
printing <b>Trump</b> doesn't commit to backing Pence for president in 2024
printing When <b>Donald Trump</b> Met Melania: The First Couple's Unusual Road to the White House
printing <b>Donald Trump</b> 'Prince of Whales' tweet prompts Wales Online change
printing Iran bomb video emerges: WATCH the top world <b>news</b> videos for today
printing Julian Castro challenges <b>Donald Trump</b>, Fox <b>News</b> in Arizona town hall
printing <b>Trump</b> Says He'd Look at Dirt given to Him by Russia, Confirms What We've Already Known :: Politics :: <b>News</b> :: <b>Donald Trump</b> :: Paste
printing Sarah Sanders leaving White House job; governor run ahead?
printing MP Mark Garnier: NHS shouldn't be forced to buy services
printing 'I Think I'd Take It:' President <b>Trump</b> And Foreign Dirt
printing Upton: New Law Will Give Veterans More Healthcare Options
printing World <b>News</b> in Brief: June 13
printing <b>

printing <b>Trump</b> Puts Saudi Arms Sales Above Khashoggi Inquiry
printing <b>Trump</b>: Surprise Question About Pence Led Him To Hesitate
printing US prods Iran for talks to ease Gulf tensions; Teheran dismisses sanctions threat, United States <b>News</b> & Top Stories
printing <b>Trump</b> says he's not seeking war as Iran warns of conflict escalation, World <b>News</b> & Top Stories
printing Kim gets letter with 'excellent content' from <b>Trump</b>, East Asia <b>News</b> & Top Stories
printing <b>Trump</b>: Surprise question about Pence led him to hesitate  WSVN 7News
printing <b>Trump</b> puts Saudi arms sales above Khashoggi inquiry  WSVN 7News
printing <b>Donald Trump</b>: US President sends excellent letter to Kim Jong-un
printing Netanyahu ally says West Bank-Gaza corridor 'irrelevant'
printing <b>Trump</b> says did not threaten to demote Federal Reserves' Powell
printing N.Korea leader receives 'excellent' letter from <b>Trump</b>: KCNA
printing North Korean leader receives

printing Xi Jinping to <b>Donald Trump</b>: China and US will 'lose by fighting'
printing <b>Trump</b>, outsider-turned-insider, sells himself as rebel for 2020 re-election bid
printing No war for oil, says  <b>Donald Trump</b>?
printing <b>Trump</b> threatens to deport millions beginning next week  Pasadena Star <b>News</b>
printing YouTuber Elijah Daniel buys town and renames it Gay Hell to troll <b>Donald Trump</b>
printing Cuba travel ban affects 500 Turks
printing <b>Donald Trump</b> Slams Fox <b>News</b> Poll, Disputes 30 Hours Title Of ABC Special  Deadline
printing How <b>Donald Trump's</b> Russia blind spot is hamstringing his administration
printing [<b>News</b> Focus] <b>Trump</b> to launch 2020 campaign in retiree-friendly Florida
printing <b>Trump</b> now a poster boy for the far-right, says London Mayor
printing Wife of former Nissan boss Carlos Ghosn seeks US President <b>Donald Trumps</b> help
printing London mayor Sadiq Khan attacks police cuts after four murders in fo

printing <b>Trump</b>, Biden trade barbs amid dueling Iowa campaign visits
printing Why Joe Biden's 2020 message is so radical
printing <b>Trump</b>, Biden trade insults during dueling Iowa visits
printing WTHolyF? FOX <b>News</b> Hysterically Whines that CNN and MSNBC 'All Seem to Be Opinion Shows'
printing Joe Biden says <b>Trump</b> destroying auto industry, but president hits back, United States <b>News</b> & Top Stories
printing China mum on Trump-Xi meeting as Huawei says sales slowing
printing Biden, <b>Trump</b> assailing each other in dueling Iowa visits  WSVN 7News
printing <b>Trump</b>, Biden head to Iowa in potential 2020 preview
printing House Resolution Would Make It Easier To Enforce Subpoenas
printing <b>Trump</b> says Fed's independence hurts him in trade war with China
printing Mexican tariffs bad, Chinese tariffs good? <b>Trumps</b> trade war lifts 'factory North America'
printing <b>Trump news</b>  live: 'First-hand accounts' of president's obstruction to be reveale

...

printing U.S. faces tough choices as it weighs next moves on Venezuela
printing Where the investigations related to President <b>Donald Trump</b> stand
printing <b>Trump</b> aides struggle to show some shutdown empathy
printing <b>Donald Trump</b> associate Roger Stone arrested, faces obstruction charge
printing Ex-Trump lawyer Michael Cohen to comply with Senate subpoena
printing Senate Subpoena For Michael Cohen; Lawyer Says Hell Comply
printing <b>Trumps</b> Shutdown Gift To Pelosi: A Unified Democratic Caucus
printing Boyd Matheson: This is about the state of the union, not the state of the government
printing Will ex-Trump lawyer Cohen comply with Senate subpoena?
printing Commerce chief asks why furloughed workers using food banks
printing Senate rejects rival Dem, GOP plans for reopening government
printing <b>Trump</b> says to give State of the Union address ‘when the shutdown is over’
printing <b>Trump</b>, GOP embrace amid political fallout from shutdown
printing <b>Donald Tr

printing Iranian Dissidents Urge <b>Trump</b> to Release Frozen Tehran Regime Assets Amid Renewed Student Protests
printing In Syria, Iran Sees a New Opportunity to Build a War Machine
printing Plan to reopen government
printing Few solutions, lots of blame
printing Presidential tweets shutdown blame, no talk of solutions
printing Israeli Minister Expresses Concern Over Fate of Kurds After <b>Trump's</b> Withdrawal From Syria
printing <b>Trump</b> blasts Fed as 'only problem' in US economy, markets down again
printing Democrats say <b>Trump</b> 'plunging country into chaos'
printing Syria envoy quits in protest
printing <b>Trump</b> shutdown showdown
printing Gov. closer to shutdown
printing Pentagon chief resigns
printing Millions donated for wall
printing Judge Emmet Sullivan hits <b>Trump</b> again--rules Administration's asylum policy "unlawful"
printing Highlights of the criminal justice overhaul bill
printing Shepard Smith Slaps Down the 'Defenders' of Flynn on FOX <b>News</b>, W

printing Abbas Praises Terrorists as Hero Martyrs, While Claiming Jerusalem Not for Sale
printing Drug Prices Going Up Despite <b>Trump</b> Promise
printing <b>Trump</b> pledges Mideast peace plan within months » Capital <b>News</b>
printing <b>Trump</b> defends Supreme Court pick but says he could change mind » Capital <b>News</b>
printing <b>Trump</b>: UN wasn't 'laughing at me' » Capital <b>News</b>
printing 'China interfering now'
printing <b>Trump</b> Awaits Meeting With Netanyahu in New York, Says Special Envoy Greenblatt
printing <b>Trump</b>, Iran's Rouhani Exchange Threats, Insults on UN's World Stage
printing <b>Trump</b> attacks Iranian leadership, boasts of accomplishments
printing PM's position on 'drug war' balanced and realistic
printing Uh-Oh: FOX <b>News</b> Host Says that <b>Trump's</b> Attack on Kavanaugh Accuser 'Might Blow This Up'
printing <b>Trump</b> drops gloves
printing At Las Vegas Rally, <b>Trump</b> Backs Kavanaugh, Treads Carefully Around Accusations
print

printing No Eagles at White House
printing <b>Trump</b> cancels Eagles visit
printing <b>Trump</b> to pardon himself?
printing N Korea's top military officials replaced
printing Insulted by U.S. tariffs
printing Trump-Kim summit back on
printing <b>Trump</b> criticizes Canada
printing Macron talks tariffs to <b>Trump</b>
printing <b>Trump</b> gets rare visit
printing G-7 officials focus on tariffs
printing <b>Trump</b> gives Martha a pass
printing Kim K thanks <b>Trump</b>
printing <b>Trump</b> escalates fight
printing Can summit be saved?
printing <b>Trump</b> confirms senior North Korean official en route to US » Capital <b>News</b>
printing <b>Trump</b>, Abe say 'imperative' to dismantle N. Korean weapons » Capital <b>News</b>
printing Trump-Kim summit coming?
printing Giuliani on spygate
printing North Korea's Kim Jong-un still 'fixed' on <b>Trump</b> summit
printing NYC's attorney general quits
printing Deal or no deal? <b>Trump</b> nears decision on Iran nuclear pact
printing <b>

printing '<b>Trump</b> is best ISIS recruiter': Clinton, Sanders round on GOP 'Islamophobia' in 3rd Democrat debate  RT USA <b>News</b>
printing Napolitano: Will the real attorney general please stand up?| Latest <b>News</b> Videos
printing Do Christians have a duty to vote for Trump?| Latest <b>News</b> Videos
printing Trump-Putin summit in Helsinki: What to know| Latest <b>News</b> Videos
printing <b>Trumps</b> NATO expectations: What to know| Latest <b>News</b> Videos
printing Ronan Farrow allegation: <b>Trump</b> had affair with Playboy model| Latest <b>News</b> Videos
printing Sunday, May 12| Latest <b>News</b> Videos
printing Josh Rosen sends message to <b>Trump</b> on <b><b>Trump</b>'s</b> golf course| Latest <b>News</b> Videos
printing Sunday, May 12| Latest <b>News</b> Videos
printing Part 3 of third presidential debate at University of Nevada
printing If You Thought Hannity Too Close To <b>Trump</b>, Check Out Pirros Corrupt Enmeshment
printing Pritzker signs trio of bills th

In [13]:
# assign list to dataframe

newsFrame = pd.DataFrame({'datePublished':date_list,'description':description_list,'keywords':keywords_list,'provider':provider_list,'title':title_list,'url':url_list})


In [18]:
newsFrame.head(200)

,datePublished,description,keywords,provider,title,url
0,2019-06-12T21:19:00,President <b>Donald Trump</b> granted ABC <b>N...,"president donald trump,donald trump news,white...",go,ABC <b>News</b> travels with President <b>Dona...,https://preview.abcnews.go.com/politics/abc-ne...
1,2019-05-22T23:00:00,<b>Donald Trump News</b> from United Press Int...,"donald trump news,photos,quotes,press",upi,<b>Donald Trump News</b>,https://www.upi.com/topic/donald_trump/?tps=1/...
2,2019-06-24T23:25:57,U.S. President <b>Donald Trump</b> has no plan...,"president donald trump,updated,news,kim",reuters,<b>Trump</b> has no plans to meet with North K...,http://feeds.reuters.com/~r/reuters/INworldNew...
3,2019-06-24T21:00:00,Both China and the United States must be willi...,"donald trump,jinping,china,news",straitstimes,"US, China must compromise to reach deal: Chine...",http://www.straitstimes.com/asia/east-asia/us-...
4,2019-06-24T20:01:08,<b>Donald Trump</b> wants a new Iran deal but ...,"donald trump,iran,news",sky,US-Iran tensions raise threat of an even grave...,https://news.sky.com/story/us-iran-tensions-ra...
5,2019-06-24T18:06:27,(Adds <b>Trump</b> quote) By Hyonhee Shin and ...,"donald trump,visit south,news",haaretz,WRAPUP 3-Trump to visit S.Korea as Pompeo rais...,http://haaretz.com/world-news/wires/1.7407257
6,2019-06-24T17:22:28,US President <b>Donald Trump</b> displays an e...,"president donald,sanctions,on iran,trump,news",algemeiner,"<b>Trump</b> Imposes New US Sanctions on Iran,...",https://www.algemeiner.com/2019/06/24/trump-im...
7,2019-06-24T17:06:57,WASHINGTON (AP) President <b>Donald Trump</b>...,"president donald trump,iran,news",pasadenastarnews,<b>Trump</b> signs order imposing sanctions on...,https://www.pasadenastarnews.com/2019/06/24/tr...
8,2019-06-24T16:42:59,US President <b>Donald Trump</b> has said he i...,"president donald,sanctions,on iran,trump,news",nigerianbulletin,New US sanctions on Iran target supreme leader...,https://www.nigerianbulletin.com/threads/new-u...
9,2019-06-24T16:25:04,<b>Trump</b> says the action follows a series ...,"president donald trump,iran,news",aljazeera,"<b>Trump</b> imposes new sanctions on Iran, ta...",https://www.aljazeera.com/news/2019/06/trump-i...


In [16]:
newsFrame.dtypes

datePublished    object
description      object
keywords         object
provider         object
title            object
url              object
dtype: object

In [17]:
newsFrame.count()

datePublished    2050
description      2050
keywords         2050
provider         2050
title            2050
url              2050
dtype: int64

In [ ]:
# import stock data 

# Load stock data into notebook as dataframe

In [ ]:
# Merge 2 DataFrames

In [ ]:
# Do sentiment analysis of news data

In [ ]:
# Use Matplotlib and stats to generate graphs and look for relationships